In [1]:
import numpy as np

from squlearn import Executor
from squlearn.encoding_circuit import *
from squlearn.kernel import FidelityKernel

## Testing single variable

In [2]:
executor = Executor("pennylane")

N_samples_x = 7
N_samples_y = 3
n_dim = 1

np.random.seed(42)
x_space = np.random.rand(N_samples_x, n_dim)
y_space = np.random.rand(N_samples_y, n_dim)

circuit1 = LayeredEncodingCircuit(1, 1)
circuit1.Ry("p")
circuit1.Rx("x")

fqk = FidelityKernel(circuit1, executor=executor, caching=False, use_expectation=True)

p1 = 0.35
fqk.assign_parameters([p1])


def manual_separable_rx_kernel(x, y, p0):
    # RxRy rotation
    # 0.375*(1 - cos(p0))**2*cos(x0 - y0) + 0.125*(1 - cos(p0))**2*cos(x0 + y0) - 0.5*(1 - cos(p0))**2 - 1.0*cos(p0) - 0.3125*cos(x0 - y0) - 0.1875*cos(x0 + y0) - 0.03125*cos(-2*p0 + x0 + y0) + 0.125*cos(-p0 + x0 + y0) + 0.375*cos(p0 - x0 + y0) + 0.375*cos(p0 + x0 - y0) + 0.125*cos(p0 + x0 + y0) + 0.03125*cos(2*p0 - x0 + y0) + 0.03125*cos(2*p0 + x0 - y0) - 0.03125*cos(2*p0 + x0 + y0) + 1.5

    return (
        0.375 * (1 - np.cos(p0)) ** 2 * np.cos(x - y)
        + 0.125 * (1 - np.cos(p0)) ** 2 * np.cos(x + y)
        - 0.5 * (1 - np.cos(p0)) ** 2
        - 1.0 * np.cos(p0)
        - 0.3125 * np.cos(x - y)
        - 0.1875 * np.cos(x + y)
        - 0.03125 * np.cos(-2 * p0 + x + y)
        + 0.125 * np.cos(-p0 + x + y)
        + 0.375 * np.cos(p0 - x + y)
        + 0.375 * np.cos(p0 + x - y)
        + 0.125 * np.cos(p0 + x + y)
        + 0.03125 * np.cos(2 * p0 - x + y)
        + 0.03125 * np.cos(2 * p0 + x - y)
        - 0.03125 * np.cos(2 * p0 + x + y)
        + 1.5
    )


def manual_separable_rx_kernel_dKdx(x, y, p0):
    """
    # RxRy rotation
    """
    return -0.25 * np.sin(x - y) + 0.125 * np.sin(2 * p0 - x + y) - 0.125 * np.sin(2 * p0 + x - y)


def manual_separable_rx_kernel_dKdp(x, y, p0):
    return 0.5 * (1 - np.cos(x - y)) * np.sin(2 * p0)


def manual_separable_rx_kernel_dKdy(x, y, p0):
    return 0.25 * np.sin(x - y) - 0.125 * np.sin(2 * p0 - x + y) + 0.125 * np.sin(2 * p0 + x - y)


def manual_separable_rx_kernel_dKdxdy(x, y, p0):
    return (
        0.375 * (1 - np.cos(p0)) ** 2 * np.cos(x - y)
        + 0.125 * (1 - np.cos(p0)) ** 2 * np.cos(x + y)
        - 0.3125 * np.cos(x - y)
        - 0.1875 * np.cos(x + y)
        - 0.03125 * np.cos(-2 * p0 + x + y)
        + 0.125 * np.cos(-p0 + x + y)
        + 0.375 * np.cos(p0 - x + y)
        + 0.375 * np.cos(p0 + x - y)
        + 0.125 * np.cos(p0 + x + y)
        + 0.03125 * np.cos(2 * p0 - x + y)
        + 0.03125 * np.cos(2 * p0 + x - y)
        - 0.03125 * np.cos(2 * p0 + x + y)
    )


def matrix_manual_todo(x_array, y_array, p0, todo):
    todo = {
        "K": manual_separable_rx_kernel,
        "dKdx": manual_separable_rx_kernel_dKdx,
        "dKdp": manual_separable_rx_kernel_dKdp,
        "dKdy": manual_separable_rx_kernel_dKdy,
        "dKdxdy": manual_separable_rx_kernel_dKdxdy,
    }[todo]
    k_matrix = np.zeros((len(x_array), len(y_array)))
    for i, x in enumerate(x_array):
        for j, y in enumerate(y_array):
            k_matrix[i, j] = todo(x[0], y[0], p0)
    return k_matrix


print(
    "Normal squlearn K == LowQNN implementation K = ",
    np.allclose(
        fqk.evaluate(x_space, y_space),
        fqk.evaluate_derivatives(x_space, y_space, values=["K"])["K"],
    ),
)
print(
    "K_analyical = K_qnn_squlearn = ",
    np.allclose(matrix_manual_todo(x_space, y_space, p1, "K"), fqk.evaluate(x_space, y_space)),
)
print(
    "dKdx_analyical = dKdx_qnn_squlearn = ",
    np.allclose(
        matrix_manual_todo(x_space, y_space, p1, "dKdx"),
        fqk.evaluate_derivatives(x_space, y_space, values=["dKdx"])["dKdx"],
    ),
)
print(
    "dKdp_analyical = dKdp_qnn_squlearn = ",
    np.allclose(
        matrix_manual_todo(x_space, y_space, p1, "dKdp"),
        fqk.evaluate_derivatives(x_space, y_space, values=["dKdp"])["dKdp"],
    ),
)
print(
    "dKdy_analyical = dKdy_qnn_squlearn = ",
    np.allclose(
        matrix_manual_todo(x_space, y_space, p1, "dKdy"),
        fqk.evaluate_derivatives(x_space, y_space, values=["dKdy"])["dKdy"],
    ),
)
print(
    "dKdxdy_analyical = dKdxdy_qnn_squlearn = ",
    np.allclose(
        matrix_manual_todo(x_space, y_space, p1, "dKdxdy"),
        fqk.evaluate_derivatives(x_space, y_space, values=["dKdxdy"])["dKdxdy"],
    ),
)

Normal squlearn K == LowQNN implementation K =  True
K_analyical = K_qnn_squlearn =  True
dKdx_analyical = dKdx_qnn_squlearn =  True
dKdp_analyical = dKdp_qnn_squlearn =  True
dKdy_analyical = dKdy_qnn_squlearn =  True
dKdxdy_analyical = dKdxdy_qnn_squlearn =  True


In [3]:
x_space = np.random.rand(1, 1)
fqk.evaluate_derivatives(x_space, x_space, values=["dKdxdy"])["dKdxdy"]

array([[0.44121055]])

# Testing multi-dimensional

In [4]:
executor = Executor("pennylane")

N_samples_x = 7
N_samples_y = 3
n_dim = 2

np.random.seed(42)
x_space = np.random.rand(N_samples_x, n_dim)
y_space = np.random.rand(N_samples_y, n_dim)

circuit1 = LayeredEncodingCircuit(n_dim, n_dim)
circuit1.Rx("p")
circuit1.Rx("x")

fqk = FidelityKernel(circuit1, executor=executor, caching=False, use_expectation=True)
p1 = 0.45
fqk.assign_parameters([p1, p1])


def manual_separable_rx_kernel(x, y, p0):
    # Rx(x0)Rx(p0) \otimes Rx(x1)Rx(p0)

    return (
        0.25 * np.cos(x[0] - y[0])
        + 0.25 * np.cos(x[1] - y[1])
        + 0.125 * np.cos(x[0] - x[1] - y[0] + y[1])
        + 0.125 * np.cos(x[0] + x[1] - y[0] - y[1])
        + 0.25
    )


def manual_separable_rx_kernel_dKdx1(x, y, p0):
    return -0.25 * (np.cos(x[1] - y[1]) + 1) * np.sin(x[0] - y[0])


def manual_separable_rx_kernel_dKdx2(x, y, p0):
    return -0.25 * (np.cos(x[0] - y[0]) + 1) * np.sin(x[1] - y[1])


def manual_separable_rx_kernel_dKdy1(x, y, p0):
    # 1.0*sin(x0/2 - y0/2)*cos(x0/2 - y0/2)*cos(x1/2 - y1/2)**2
    return (
        1.0
        * np.sin(x[0] / 2 - y[0] / 2)
        * np.cos(x[0] / 2 - y[0] / 2)
        * np.cos(x[1] / 2 - y[1] / 2) ** 2
    )


def manual_separable_rx_kernel_dKdy2(x, y, p0):
    return (
        1.0
        * np.sin(x[1] / 2 - y[1] / 2)
        * np.cos(x[0] / 2 - y[0] / 2) ** 2
        * np.cos(x[1] / 2 - y[1] / 2)
    )


def matrix_manual_todo(x_array, y_array, p0, todo):
    todo = {
        "K": manual_separable_rx_kernel,
        "dKdx1": manual_separable_rx_kernel_dKdx1,
        "dKdx2": manual_separable_rx_kernel_dKdx2,
        "dKdy1": manual_separable_rx_kernel_dKdy1,
        "dKdy2": manual_separable_rx_kernel_dKdy2,
    }[todo]
    k_matrix = np.zeros((len(x_array), len(y_array)))
    for i, x in enumerate(x_array):
        for j, y in enumerate(y_array):
            k_matrix[i, j] = todo(x, y, p0)
    return k_matrix


print(
    "Normal squlearn K == LowQNN implementation K = ",
    np.allclose(
        fqk.evaluate(x_space, y_space),
        fqk.evaluate_derivatives(x_space, y_space, values=["K"])["K"],
    ),
)
print(
    "K_analyical = K_qnn_squlearn = ",
    np.allclose(matrix_manual_todo(x_space, y_space, p1, "K"), fqk.evaluate(x_space, y_space)),
)
print(
    "dKdx1_analyical = dKdx_qnn_squlearn = ",
    np.allclose(
        matrix_manual_todo(x_space, y_space, p1, "dKdx1"),
        fqk.evaluate_derivatives(x_space, y_space, values=["dKdx"])["dKdx"][0],
    ),
)
print(
    "dKdx2_analyical = dKdx_qnn_squlearn = ",
    np.allclose(
        matrix_manual_todo(x_space, y_space, p1, "dKdx2"),
        fqk.evaluate_derivatives(x_space, y_space, values=["dKdx"])["dKdx"][1],
    ),
)
print(
    "dKdy1_analyical = dKdy_qnn_squlearn = ",
    np.allclose(
        matrix_manual_todo(x_space, y_space, p1, "dKdy1"),
        fqk.evaluate_derivatives(x_space, y_space, values=["dKdy"])["dKdy"][0],
    ),
)
print(
    "dKdy2_analyical = dKdy_qnn_squlearn = ",
    np.allclose(
        matrix_manual_todo(x_space, y_space, p1, "dKdy2"),
        fqk.evaluate_derivatives(x_space, y_space, values=["dKdy"])["dKdy"][1],
    ),
)

Normal squlearn K == LowQNN implementation K =  True
K_analyical = K_qnn_squlearn =  True
dKdx1_analyical = dKdx_qnn_squlearn =  True
dKdx2_analyical = dKdx_qnn_squlearn =  True
dKdy1_analyical = dKdy_qnn_squlearn =  True
dKdy2_analyical = dKdy_qnn_squlearn =  True


In [5]:
x_space = np.random.rand(1, 2)
fqk.evaluate_derivatives(x_space, x_space, values=["dKdx"])

{'x': array([[0.61185289, 0.13949386, 0.61185289, 0.13949386]]),
 'param': [0.45, 0.45],
 'param_op': array([0.25, 0.25, 0.25, 0.25]),
 'dKdx': array([[[0.00000000e+00]],
 
        [[1.38777878e-17]]])}

In [12]:
import numpy as np
import sympy as sp
from squlearn import Executor
from squlearn.encoding_circuit import LayeredEncodingCircuit
from squlearn.kernel.lowlevel_kernel import FidelityKernel

def setup_single_variable():
    x, y, p = sp.symbols("x y p")
    sympy_K = 0.375 * (1 - sp.cos(p)) ** 2 * sp.cos(x - y) + 0.125 * (1 - sp.cos(p)) ** 2 * sp.cos(x + y) - 0.5 * (1 - sp.cos(p)) ** 2 - 1.0 * sp.cos(p) - 0.3125 * sp.cos(x - y) - 0.1875 * sp.cos(x + y) - 0.03125 * sp.cos(-2 * p + x + y) + 0.125 * sp.cos(-p + x + y) + 0.375 * sp.cos(p - x + y) + 0.375 * sp.cos(p + x - y) + 0.125 * sp.cos(p + x + y) + 0.03125 * sp.cos(2 * p - x + y) + 0.03125 * sp.cos(2 * p + x - y) - 0.03125 * sp.cos(2 * p + x + y) + 1.5

    sympy_values = {
        "K": sympy_K,
        "dKdx": sp.diff(sympy_K, x),
        "dKdy": sp.diff(sympy_K, y),
        "dKdxdx": sp.diff(sp.diff(sympy_K, x), x),
        "dKdp": sp.diff(sympy_K, p),
    }

    return x, y, p, sympy_values

def setup_multi_variable():
    x0, y0, p0 = sp.symbols("x0 y0 p0")
    x1, y1, p1 = sp.symbols("x1 y1 p1")

    sympy_K = 0.25*sp.cos(x0 - y0) + 0.25*sp.cos(x1 - y1) + 0.125*sp.cos(x0 - x1 - y0 + y1) + 0.125*sp.cos(x0 + x1 - y0 - y1) + 0.25
    
    sympy_values = {
        "K": sympy_K,
        "dKdx0": sp.diff(sympy_K, x0),
        "dKdx1": sp.diff(sympy_K, x1),
        "dKdy0": sp.diff(sympy_K, y0),
        "dKdy1": sp.diff(sympy_K, y1),
        "dKdp0": sp.diff(sympy_K, p0),
        "dKdp1": sp.diff(sympy_K, p1),
    }

    return x0, y0, x1, y1, p0, p1, sympy_values

def create_fidelity_kernel(num_features, initial_parameters):
    executor = Executor()
    encoding_circuit = LayeredEncodingCircuit(
        num_qubits=num_features, num_features=num_features
    )

    if num_features == 1:
        encoding_circuit.Ry("p") #For 1 feature, the FQK analytical kernel is:  Rx(x)@Ry(p)
    elif num_features == 2:
        encoding_circuit.Rx("p") #For 2 features, the FQK analytical kernel is:  Rx(x0)@Ry(p0) x Rx(x1)@Ry(p1)
    encoding_circuit.Rx("x")
    
    kernel = FidelityKernel(encoding_circuit, executor=executor, caching=False, use_expectation=True, initial_parameters=initial_parameters)    

    return kernel

def test_single_variable_derivatives():
    x, y, p, sympy_values = setup_single_variable()

    x_num, y_num = 0.79, -0.31
    p_num = -0.63

    subs = {x: x_num, y: y_num, p: p_num}
    sympy_num_values = {key: sympy_values[key].evalf(subs=subs) for key in sympy_values}

    kernel = create_fidelity_kernel(1, [p_num])

    values = kernel.evaluate_derivatives(
        np.array([[x_num]]), np.array([[y_num]]), ["K", "dKdx", "dKdy", "dKdxdx", "dKdp"]
    )
    for key in ["K", "dKdx", "dKdy", "dKdxdx", "dKdp"]:
        assert np.allclose(
            np.array(values[key]).flatten().astype(float),
            np.array(sympy_num_values[key]).astype(float),
            atol=1e-7,
        )
    print("Single variable derivatives test passed.")

def test_multi_variable_derivatives():
    x0, y0, x1, y1, p0, p1, sympy_values = setup_multi_variable()

    x0_num, y0_num = 0.79, -0.31
    x1_num, y1_num = 0.9, -1.31
    p_num = -0.63

    subs = {
        x0: x0_num,
        y0: y0_num,
        x1: x1_num,
        y1: y1_num,
        p0: p_num,
        p1: p_num,
    }
    sympy_num_values = {
        "K": sympy_values["K"].evalf(subs=subs),
        "dKdx": [
            sympy_values["dKdx0"].evalf(subs=subs),
            sympy_values["dKdx1"].evalf(subs=subs),
        ],
        "dKdy": [
            sympy_values["dKdy0"].evalf(subs=subs),
            sympy_values["dKdy1"].evalf(subs=subs),
        ],
        "dKdp": [
            sympy_values["dKdp0"].evalf(subs=subs),
            sympy_values["dKdp1"].evalf(subs=subs),
        ],
    }

    kernel = create_fidelity_kernel(2, [p_num, p_num])

    values = kernel.evaluate_derivatives(
        np.array([[x0_num, x1_num]]), np.array([[y0_num, y1_num]]), ["K", "dKdx", "dKdy", "dKdp"]
    )

    for key in ["K", "dKdx", "dKdy", "dKdp"]:
        assert np.allclose(
            np.array(values[key]).flatten().astype(float),
            np.array(sympy_num_values[key]).astype(float),
            atol=1e-7,
        )
    print("Multi variable derivatives test passed.")

# Run the tests
#


In [13]:
test_single_variable_derivatives()

kernel [-0.63]
kernel <squlearn.kernel.lowlevel_kernel.fidelity_kernel.FidelityKernel object at 0x000002581CD94DC0>
Single variable derivatives test passed.


In [15]:
test_multi_variable_derivatives()

Multi variable derivatives test passed.
